# Optimisation: XNES

This example shows you how to run a global optimisation with XNES.

For a more elaborate example of an optimisation, see: https://github.com/martinjrobins/pints/blob/master/examples/optimisation-first-example.ipynb

In [5]:
from __future__ import print_function
import pints
import pints.toy as toy
import numpy as np
import matplotlib.pyplot as pl

# Load a forward model
model = toy.LogisticModel()

# Create some toy data
real_parameters = [0.015, 500]
times = np.linspace(0, 1000, 1000)
values = model.simulate(real_parameters, times)

# Create an object with links to the model and time series
problem = pints.SingleSeriesProblem(model, times, values)

# Select a score function
score = pints.SumOfSquaresError(problem)

# Select some boundaries
boundaries = pints.Boundaries([0, 400], [0.03, 600])

# Perform an optimization with boundaries and hints
x0 = 0.011, 400
sigma0 = [0.01, 100]
found_parameters, found_value = pints.xnes(
    score,
    boundaries,
    x0,
    sigma0,
    )

print('Score at true solution: ')
print(score(real_parameters))

print('Found solution:          True parameters:' )
for k, x in enumerate(found_parameters):
    print(pints.strfloat(x) + '    ' + pints.strfloat(real_parameters[k]))

Running in parallel mode with population size 8
1: 2168581.38797
2: 1265352.02018
3: 1265352.02018
20: 378.661004652
40: 0.028823319486
60: 2.27026057436e-06
80: 7.34613987428e-09
100: 5.10759549154e-14
120: 1.69690351827e-18
140: 7.6529340366e-23
160: 0.0
180: 0.0
Halting: No significant change for 100 iterations.
191: 0.0
Score at true solution: 
0.0
Found solution:          True parameters:
 1.49999999999999994e-02     1.49999999999999994e-02
 5.00000000000000000e+02     5.00000000000000000e+02
